In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

plt.rcParams['font.family'] = 'NanumGothic'

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
minor_s = pd.read_csv('./drive/MyDrive/공유문서함/상장csv/240317상장비주력_stand.csv', index_col=0)
minor_test = pd.read_csv('./drive/MyDrive/공유문서함/상장csv/240317_상장비주력test.csv', index_col=0)

In [ ]:
# major_s.drop(['회사명',
#   '회계년도',
#   '설립일',
#   '상장일',
#   '상장폐지일',
#   '상장협 산업분류(소분류)_x',
#   '상장협 산업분류(중분류)_x',
#   '회계년',
#   'Unnamed: 0'], axis=1, inplace=True)

In [ ]:
# major_test.drop(['회사명',
#   '회계년도',
#   '설립일',
#   '상장일',
#   '상장폐지일',
#   '상장협 산업분류(소분류)_x',
#   '상장협 산업분류(중분류)_x',
#   '회계년',
#   'Unnamed: 0'], axis=1, inplace=True)

##### train_df, test_df 설정

In [ ]:
train_df = minor_s[['CASH FLOW 대 매출액비율',
                    '매출액순이익률',
                    '1주당매출액',
                    '자기자본순이익률',
                    '적립금비율(재정비율)',
                    '자본',
                    '재고자산 대 유동자산비율',
                    '매출채권회전기간',
                    '비유동부채비율',
                    '순운전자본회전률',
                    '유보율',
                    'CASH FLOW 대 차입금비율',
                    '유동부채비율',
                    '부실']]

In [ ]:
test_df = minor_test[['CASH FLOW 대 매출액비율',
                    '매출액순이익률',
                    '1주당매출액',
                    '자기자본순이익률',
                    '적립금비율(재정비율)',
                    '자본',
                    '재고자산 대 유동자산비율',
                    '매출채권회전기간',
                    '비유동부채비율',
                    '순운전자본회전률',
                    '유보율',
                    'CASH FLOW 대 차입금비율',
                    '유동부채비율',
                    '부실']]

In [ ]:
train_df.reset_index(drop=True,inplace=True)

In [ ]:
x_train = train_df.drop("부실",axis=1)
y_train = train_df["부실"]
x_test =test_df.drop("부실",axis=1)
y_test = test_df["부실"]

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE

In [ ]:
from imblearn.under_sampling import RandomUnderSampler

# RandomUnderSampler 객체 생성 및 비율 조정
ratio = {0: 264, 1: 88}  # 클래스 0과 클래스 1의 샘플 수를 동일하게 설정
under_sampler = RandomUnderSampler(sampling_strategy=ratio)

# RandomUnderSampler를 사용하여 샘플링된 특징 및 타겟 데이터 생성
x_train, y_train = under_sampler.fit_resample(x_train, y_train)

# 샘플링된 타겟 데이터의 클래스별 샘플 수 확인
print(y_train.value_counts())

0.0    264
1.0     88
Name: 부실, dtype: int64


In [ ]:
# imbalanced-learn 패키지
from imlearn.over_sampling import SMOTE

# 검증 데이터나 테스트 데이터가 아닌 학습데이터에서만 오버샘플링 사용할 것
smote = SMOTE(random_state=11)
X_train_over, y_train_over = smote.fit_sample(X_train, y_train)

In [ ]:
# x_train, y_train = RandomOverSampler(random_state=4).fit_resample(x_train,y_train)
# print(x_train.shape)
# y_train.value_counts()

(9338, 13)


1.0    4669
0.0    4669
Name: 부실, dtype: int64

In [ ]:
# 성능확인 코드
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.metrics import f1_score, roc_auc_score, average_precision_score

def get_clf_eval(y_test, pred):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    roc_score = roc_auc_score(y_test, pred)
    pr_score = average_precision_score(y_test, pred)
    f1 = f1_score(y_test, pred)
    print('오차행렬')
    print(confusion)
    print('정확도: {0:.4f}, 정밀도 : {1:.4f}, 재현율:{2:.4f},F1 스코어:{3:.4f}'.format(accuracy, precision, recall, f1))
    print('ROC 스코어: {0:.4f}, PR score : {1:.4f}'.format(roc_score, pr_score))

##### 1. RandomForest

In [ ]:
# 랜덤포레스트 구현
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier(random_state=52)

rf_clf.fit(x_train, y_train)
rf_pred = rf_clf.predict(x_test)
rf_pred_proba = rf_clf.predict_proba(x_test)

In [ ]:
print("랜덤포레스트")
get_clf_eval(y_test, rf_pred)

랜덤포레스트
오차행렬
[[1557  174]
 [  16    3]]
정확도: 0.8914, 정밀도 : 0.0169, 재현율:0.1579,F1 스코어:0.0306
ROC 스코어: 0.5287, PR score : 0.0118


##### 하이퍼파라미터 조정 & 임계값 설정한 뒤 모델링

In [ ]:
# RandomizedSearchCV를 사용하여 최적의 하이퍼파라미터 찾기
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
rf_clf = RandomForestClassifier(random_state=42)
random_search = RandomizedSearchCV(rf_clf, param_distributions=param_grid, n_iter=10, cv=5, scoring='accuracy', random_state=42)
random_search.fit(x_train, y_train)

# 최적의 모델과 하이퍼파라미터 가져오기
best_rf_clf = random_search.best_estimator_

# 예측 확률 계산
rf_pred_proba = best_rf_clf.predict_proba(x_test)

# # 임계값 설정
# threshold = 0.1
# rf_pred = (rf_pred_proba[:, 1] > threshold).astype(int)

# # 성능 평가
# print("랜덤포레스트")
# get_clf_eval(y_test, rf_pred)

In [ ]:
# 임계값 설정
threshold = 0.6
rf_pred = (rf_pred_proba[:, 1] > threshold).astype(int)

# 성능 평가
print("랜덤포레스트")
get_clf_eval(y_test, rf_pred)

랜덤포레스트
오차행렬
[[1516  215]
 [   9   10]]
정확도: 0.8720, 정밀도 : 0.0444, 재현율:0.5263,F1 스코어:0.0820
ROC 스코어: 0.7011, PR score : 0.0285


###### 임곗값 설정

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# 임계값 범위 설정
thresholds = [0.1, 0.2, 0.4, 0.45,0.5]

for threshold in thresholds:
    # 예측 확률을 이진 예측으로 변환
    binary_predictions = (rf_pred_proba[:, 1] > threshold).astype(int)

    # 성능 지표 계산
    accuracy = accuracy_score(y_test, binary_predictions)
    precision = precision_score(y_test, binary_predictions)
    recall = recall_score(y_test, binary_predictions)
    f1 = f1_score(y_test, binary_predictions)

    # 결과 출력
    print(f"임곗값: {threshold:.2f}")
    print(f"정확도: {accuracy:.4f}, 정밀도: {precision:.4f}, 재현율: {recall:.4f}, F1 점수: {f1:.4f}")

임곗값: 0.10
정확도: 0.0109, 정밀도: 0.0109, 재현율: 1.0000, F1 점수: 0.0215
임곗값: 0.20
정확도: 0.5863, 정밀도: 0.0256, 재현율: 1.0000, F1 점수: 0.0499
임곗값: 0.40
정확도: 0.6703, 정밀도: 0.0271, 재현율: 0.8421, F1 점수: 0.0525
임곗값: 0.45
정확도: 0.6766, 정밀도: 0.0276, 재현율: 0.8421, F1 점수: 0.0535
임곗값: 0.50
정확도: 0.7240, 정밀도: 0.0285, 재현율: 0.7368, F1 점수: 0.0548


###### 하이퍼파라미터조정

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# 랜덤 포레스트 분류기
rf_clf = RandomForestClassifier()

# 탐색할 하이퍼파라미터 공간 설정
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# 랜덤 서치 객체 생성
random_search = RandomizedSearchCV(rf_clf, param_distributions=param_grid, n_iter=10, cv=5, scoring='accuracy', random_state=52)

# 랜덤 서치 실행
random_search.fit(x_train, y_train)

# 최적의 하이퍼파라미터 출력
print("최적의 하이퍼파라미터:", random_search.best_params_)

# 최적의 하이퍼파라미터를 사용하여 모델 평가
best_rf_clf = random_search.best_estimator_
rf_pred = best_rf_clf.predict(x_test)

# 예측 결과 출력
print("테스트 데이터 예측 결과:")
print(classification_report(y_test, rf_pred))

최적의 하이퍼파라미터: {'n_estimators': 100, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_depth': None}
테스트 데이터 예측 결과:
              precision    recall  f1-score   support

         0.0       1.00      0.69      0.81      1731
         1.0       0.03      0.84      0.06        19

    accuracy                           0.69      1750
   macro avg       0.51      0.77      0.44      1750
weighted avg       0.99      0.69      0.81      1750



In [ ]:
# 하이퍼파라미터 조정 후, 랜덤포레스트
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier(n_estimators=100, min_samples_split= 2, min_samples_leaf= 2, max_depth=None, random_state=52)

rf_clf.fit(x_train, y_train)
rf_pred = rf_clf.predict(x_test)
rf_pred_proba = rf_clf.predict_proba(x_test)

In [ ]:
print("랜덤포레스트")
get_clf_eval(y_test, rf_pred)

랜덤포레스트
오차행렬
[[1253  478]
 [   5   14]]
정확도: 0.7240, 정밀도 : 0.0285, 재현율:0.7368,F1 스코어:0.0548
ROC 스코어: 0.7304, PR score : 0.0238


##### 2. LogisticRegression

In [ ]:
# 로지스틱 회귀 모델
from sklearn.linear_model import LogisticRegression
lr_clf = LogisticRegression()
lr_clf.fit(x_train, y_train)
lr_pred = lr_clf.predict(x_test)
lr_pred_proba = lr_clf.predict_proba(x_test)

In [ ]:
print("로지스틱")
get_clf_eval(y_test, lr_pred)

로지스틱
오차행렬
[[1728    3]
 [  17    2]]
정확도: 0.9886, 정밀도 : 0.4000, 재현율:0.1053,F1 스코어:0.1667
ROC 스코어: 0.5518, PR score : 0.0518


In [ ]:
# RandomizedSearchCV를 사용하여 최적의 하이퍼파라미터 찾기
param_grid = {
    'C': [0.001, 0.01, 0.1, 0.2, 0.3, 0.5, 1.0, 10.0],
    'penalty': ['l1', 'l2']
}
logistic_clf = LogisticRegression()
random_search = RandomizedSearchCV(logistic_clf, param_distributions=param_grid, n_iter=10, cv=5, scoring='accuracy', random_state=42)
random_search.fit(x_train, y_train)

# 최적의 모델과 하이퍼파라미터 가져오기
best_logistic_clf = random_search.best_estimator_

# 예측 확률 계산
logistic_pred_proba = best_logistic_clf.predict_proba(x_test)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

# 그리드서치를 위한 하이퍼파라미터 그리드 설정
param_grid = {
    'C': [0.001, 0.01, 0.1, 0.2, 0.3, 0.5, 1.0, 10.0],
    'penalty': ['l1', 'l2']
}

# 로지스틱 회귀 모델 생성
logistic_clf = LogisticRegression()

# 그리드서치 수행
grid_search = GridSearchCV(logistic_clf, param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(x_train, y_train)

# 최적의 모델과 하이퍼파라미터 가져오기
best_logistic_clf = grid_search.best_estimator_

# 예측 확률 계산
logistic_pred_proba = best_logistic_clf.predict_proba(x_test)

In [ ]:
# 임계값 설정 (예시로 0.5 사용)
threshold = 0.8
logistic_pred = (logistic_pred_proba[:, 1] > threshold).astype(int)

# 성능 평가
print("로지스틱")
get_clf_eval(y_test, logistic_pred)

로지스틱
오차행렬
[[1728    3]
 [  17    2]]
정확도: 0.9886, 정밀도 : 0.4000, 재현율:0.1053,F1 스코어:0.1667
ROC 스코어: 0.5518, PR score : 0.0518


###### 임곗값 설정

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# 임계값 범위 설정
thresholds = [0.01, 1.0, 0.5, 0.5, 0.6]

for threshold in thresholds:
    # 예측 확률을 이진 예측으로 변환
    binary_predictions = (lr_pred_proba[:, 1] > threshold).astype(int)

    # 성능 지표 계산
    accuracy = accuracy_score(y_test, binary_predictions)
    precision = precision_score(y_test, binary_predictions)
    recall = recall_score(y_test, binary_predictions)
    f1 = f1_score(y_test, binary_predictions)

    # 결과 출력
    print(f"임곗값: {threshold:.2f}")
    print(f"정확도: {accuracy:.4f}, 정밀도: {precision:.4f}, 재현율: {recall:.4f}, F1 점수: {f1:.4f}")

임곗값: 0.01
정확도: 0.9886, 정밀도: 0.4000, 재현율: 0.1053, F1 점수: 0.1667
임곗값: 1.00
정확도: 0.9891, 정밀도: 0.0000, 재현율: 0.0000, F1 점수: 0.0000
임곗값: 0.50
정확도: 0.9886, 정밀도: 0.4000, 재현율: 0.1053, F1 점수: 0.1667
임곗값: 0.50
정확도: 0.9886, 정밀도: 0.4000, 재현율: 0.1053, F1 점수: 0.1667
임곗값: 0.60
정확도: 0.9886, 정밀도: 0.4000, 재현율: 0.1053, F1 점수: 0.1667


###### 하이퍼파라미터 조정

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# 로지스틱 회귀 모델
logistic_clf = LogisticRegression()

# 탐색할 하이퍼파라미터 공간 설정
param_grid = {
    'C': [0.001, 0.01, 0.1, 1.0, 10.0],
    'penalty': ['l1', 'l2']
}

# 랜덤 서치 객체 생성
random_search = RandomizedSearchCV(logistic_clf, param_distributions=param_grid, n_iter=10, cv=5, scoring='accuracy', random_state=42)

# 랜덤 서치 실행
random_search.fit(x_train, y_train)

# 최적의 하이퍼파라미터 출력
print("최적의 하이퍼파라미터:", random_search.best_params_)

# 최적의 하이퍼파라미터를 사용하여 모델 평가
best_logistic_clf = random_search.best_estimator_
logistic_pred = best_logistic_clf.predict(x_test)

# 예측 결과 출력
print("테스트 데이터 예측 결과:")
print(classification_report(y_test, logistic_pred))

최적의 하이퍼파라미터: {'penalty': 'l2', 'C': 1.0}
테스트 데이터 예측 결과:
              precision    recall  f1-score   support

         0.0       0.99      1.00      0.99      1731
         1.0       0.40      0.11      0.17        19

    accuracy                           0.99      1750
   macro avg       0.70      0.55      0.58      1750
weighted avg       0.98      0.99      0.99      1750



In [ ]:
# 하이퍼파라미터 조정 후, 로지스틱 회귀 모델
from sklearn.linear_model import LogisticRegression

lr_clf = LogisticRegression(penalty='l2', C=0.01)
lr_clf.fit(x_train, y_train)
lr_pred = lr_clf.predict(x_test)
lr_pred_proba = lr_clf.predict_proba(x_test)

In [ ]:
print("로지스틱")
get_clf_eval(y_test, lr_pred)

로지스틱
오차행렬
[[1728    3]
 [  17    2]]
정확도: 0.9886, 정밀도 : 0.4000, 재현율:0.1053,F1 스코어:0.1667
ROC 스코어: 0.5518, PR score : 0.0518


##### 3. SVM

In [ ]:
# SVM 모형 구현
from sklearn import svm

svm_clf = svm.SVC(kernel='linear', probability=True)  # 선형 SVM 모델 생성
svm_clf.fit(x_train, y_train)  # 훈련 데이터를 이용하여 모델 훈련
# 테스트 데이터를 사용하여 예측
svm_pred = svm_clf.predict(x_test)
svm_pred_proba = svm_clf.predict_proba(x_test)

In [ ]:
print("SVM")
get_clf_eval(y_test, svm_pred)

SVM
오차행렬
[[1728    3]
 [  17    2]]
정확도: 0.9886, 정밀도 : 0.4000, 재현율:0.1053,F1 스코어:0.1667
ROC 스코어: 0.5518, PR score : 0.0518


In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report

# RandomizedSearchCV를 사용하여 최적의 하이퍼파라미터 찾기
param_grid = {
    'C': [0.001, 0.01, 0.1, 1.0, 10.0],
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': ['scale', 'auto']
}
svm_clf = SVC(probability=True)
random_search = RandomizedSearchCV(svm_clf, param_distributions=param_grid, n_iter=10, cv=5, scoring='accuracy', random_state=42)
random_search.fit(x_train, y_train)

# 최적의 모델과 하이퍼파라미터 가져오기
best_svm_clf = random_search.best_estimator_

# 예측 확률 계산
svm_pred_proba = best_svm_clf.predict_proba(x_test)

In [ ]:
# 임계값 설정 (예시로 0.5 사용)
threshold = 0.3
svm_pred = (svm_pred_proba[:, 1] > threshold).astype(int)

# 성능 평가
print("SVM")
get_clf_eval(y_test, svm_pred)

SVM
오차행렬
[[1728    3]
 [  17    2]]
정확도: 0.9886, 정밀도 : 0.4000, 재현율:0.1053,F1 스코어:0.1667
ROC 스코어: 0.5518, PR score : 0.0518


###### 임곗값 조정

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# 임계값 범위 설정
thresholds = [0.3, 0.4, 0.5, 0.6]

for threshold in thresholds:
    # 예측 확률을 이진 예측으로 변환
    binary_predictions = (svm_pred_proba[:, 1] > threshold).astype(int)

    # 성능 지표 계산
    accuracy = accuracy_score(y_test, binary_predictions)
    precision = precision_score(y_test, binary_predictions)
    recall = recall_score(y_test, binary_predictions)
    f1 = f1_score(y_test, binary_predictions)

    # 결과 출력
    print(f"임곗값: {threshold:.2f}")
    print(f"정확도: {accuracy:.4f}, 정밀도: {precision:.4f}, 재현율: {recall:.4f}, F1 점수: {f1:.4f}")

임곗값: 0.30
정확도: 0.9886, 정밀도: 0.4000, 재현율: 0.1053, F1 점수: 0.1667
임곗값: 0.40
정확도: 0.9886, 정밀도: 0.4000, 재현율: 0.1053, F1 점수: 0.1667
임곗값: 0.50
정확도: 0.9886, 정밀도: 0.4000, 재현율: 0.1053, F1 점수: 0.1667
임곗값: 0.60
정확도: 0.9886, 정밀도: 0.4000, 재현율: 0.1053, F1 점수: 0.1667


###### 하이퍼파라미터 조정

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report

# SVM 모델
svm_clf = SVC()

# 탐색할 하이퍼파라미터 공간 설정
param_grid = {
    'C': [0.001, 0.01, 0.1, 1.0, 10.0],
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': ['scale', 'auto']
}

# 랜덤 서치 객체 생성
random_search = RandomizedSearchCV(svm_clf, param_distributions=param_grid, n_iter=10, cv=5, scoring='accuracy', random_state=42)

# 랜덤 서치 실행
random_search.fit(x_train, y_train)

# 최적의 하이퍼파라미터 출력
print("최적의 하이퍼파라미터:", random_search.best_params_)

# 최적의 하이퍼파라미터를 사용하여 모델 평가
best_svm_clf = random_search.best_estimator_
svm_pred = best_svm_clf.predict(x_test)

# 예측 결과 출력
print("테스트 데이터 예측 결과:")
print(classification_report(y_test, svm_pred))


최적의 하이퍼파라미터: {'kernel': 'rbf', 'gamma': 'auto', 'C': 10.0}
테스트 데이터 예측 결과:
              precision    recall  f1-score   support

         0.0       0.99      1.00      0.99      1731
         1.0       0.00      0.00      0.00        19

    accuracy                           0.99      1750
   macro avg       0.49      0.50      0.50      1750
weighted avg       0.98      0.99      0.98      1750



In [ ]:
# 하이퍼파라미터 조정 후, SVM 모형 구현
from sklearn import svm

svm_clf = svm.SVC(kernel='rbf', gamma= 'auto', C= 10, probability=True)  # 선형 SVM 모델 생성
svm_clf.fit(x_train, y_train)  # 훈련 데이터를 이용하여 모델 훈련
# 테스트 데이터를 사용하여 예측
svm_pred = svm_clf.predict(x_test)
svm_pred_proba = svm_clf.predict_proba(x_test)

In [ ]:
print("SVM")
get_clf_eval(y_test, svm_pred)

SVM
오차행렬
[[1731    0]
 [  19    0]]
정확도: 0.9891, 정밀도 : 0.0000, 재현율:0.0000,F1 스코어:0.0000
ROC 스코어: 0.5000, PR score : 0.0109


##### 4. KNN

In [ ]:
# KNN 모형 구현
from sklearn.neighbors import KNeighborsClassifier

knn_clf = KNeighborsClassifier(n_neighbors=5)
knn_clf.fit(x_train, y_train)
knn_pred = knn_clf.predict(x_test)
knn_pred_proba = knn_clf.predict_proba(x_test)

In [ ]:
print("knn")
get_clf_eval(y_test, knn_pred)

knn
오차행렬
[[1728    3]
 [  17    2]]
정확도: 0.9886, 정밀도 : 0.4000, 재현율:0.1053,F1 스코어:0.1667
ROC 스코어: 0.5518, PR score : 0.0518


In [ ]:
# RandomizedSearchCV를 사용하여 최적의 하이퍼파라미터 찾기
param_grid = {
    'n_neighbors': [1, 3, 5, 7, 9, 11, 13],
    'weights': ['uniform', 'distance'],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'p': [1, 2]  # 1은 맨해튼 거리, 2는 유클리디언 거리
}
knn_clf = KNeighborsClassifier()
random_search = RandomizedSearchCV(knn_clf, param_distributions=param_grid, n_iter=10, cv=5, scoring='accuracy', random_state=42)
random_search.fit(x_train, y_train)

# 최적의 모델과 하이퍼파라미터 가져오기
best_knn_clf = random_search.best_estimator_

# 예측 확률 계산
knn_pred_proba = best_knn_clf.predict_proba(x_test)

In [ ]:
# 임계값 설정
threshold = 0.2
knn_pred = (knn_pred_proba[:, 1] > threshold).astype(int)

# 성능 평가
print("KNN")
get_clf_eval(y_test, knn_pred)

KNN
오차행렬
[[1306  425]
 [   2   17]]
정확도: 0.7560, 정밀도 : 0.0385, 재현율:0.8947,F1 스코어:0.0738
ROC 스코어: 0.8246, PR score : 0.0356


##### 5. LGBM

In [ ]:
# LGBM 모형 구현
import lightgbm as lgb

lgb_model = lgb.LGBMClassifier()
lgb_model.fit(x_train, y_train)
# 테스트 데이터를 사용하여 예측
lgb_pred = lgb_model.predict(x_test)
lgb_pred_proba = lgb_model.predict_proba(x_test)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 88, number of negative: 264
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000550 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3234
[LightGBM] [Info] Number of data points in the train set: 352, number of used features: 28
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.250000 -> initscore=-1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain,

In [ ]:
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMClassifier

# 그리드서치를 위한 하이퍼파라미터 그리드 설정
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7],
    'min_child_weight': [1, 3, 5]
}

# LGBMClassifier 모델 생성
lgbm_clf = LGBMClassifier()

# 그리드서치 수행
grid_search = GridSearchCV(lgbm_clf, param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(x_train, y_train)

# 최적의 모델과 하이퍼파라미터 가져오기
best_lgbm_clf = grid_search.best_estimator_

# 예측 확률 계산
lgbm_pred = best_lgbm_clf.predict(x_test)
lgbm_pred_proba = best_lgbm_clf.predict_proba(x_test)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best 

Exception ignored on calling ctypes callback function: <function _log_callback at 0x7f81d694cca0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/lightgbm/basic.py", line 203, in _log_callback
    def _log_callback(msg: bytes) -> None:
KeyboardInterrupt: 


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best 

In [ ]:
# 임계값 설정
threshold = 0.2
lgb_pred = (lgb_pred_proba[:, 1] > threshold).astype(int)

# 성능 평가
print("LGBM")
get_clf_eval(y_test, lgb_pred)

LGBM
오차행렬
[[1259  472]
 [   2   17]]
정확도: 0.7291, 정밀도 : 0.0348, 재현율:0.8947,F1 스코어:0.0669
ROC 스코어: 0.8110, PR score : 0.0322


##### 6. xgboost

In [ ]:
# xgboost 모형 구현
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
import xgboost as xgb ## XGBoost 불러오기
from xgboost import plot_importance

xgb_wrapper = XGBClassifier()

xgb_wrapper.fit(x_train, y_train, verbose = True)


xgb_pred = xgb_wrapper.predict(x_test)
xgb_pred_proba = xgb_wrapper.predict_proba(x_test)

In [ ]:
print("xgb")
get_clf_eval(y_test, xgb_pred)

xgb
오차행렬
[[1386  345]
 [   8   11]]
정확도: 0.7983, 정밀도 : 0.0309, 재현율:0.5789,F1 스코어:0.0587
ROC 스코어: 0.6898, PR score : 0.0225


In [ ]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier

# 그리드서치를 위한 하이퍼파라미터 그리드 설정
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7],
    'min_child_weight': [1, 3, 5]
}

# XGBClassifier 모델 생성
xgb_clf = XGBClassifier()

# 그리드서치 수행
grid_search = GridSearchCV(xgb_clf, param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(x_train, y_train)

# 최적의 모델과 하이퍼파라미터 가져오기
best_xgb_clf = grid_search.best_estimator_

# 테스트 데이터에 대한 예측 수행
xgb_pred = best_xgb_clf.predict(x_test)
xgb_pred_proba = best_xgb_clf.predict_proba(x_test)

In [ ]:
# 임계값 설정
threshold = 0.2
xgb_pred = (xgb_pred_proba[:, 1] > threshold).astype(int)

# 성능 평가
print("xgboost")
get_clf_eval(y_test, lgb_pred)

xgboost
오차행렬
[[1259  472]
 [   2   17]]
정확도: 0.7291, 정밀도 : 0.0348, 재현율:0.8947,F1 스코어:0.0669
ROC 스코어: 0.8110, PR score : 0.0322


#### 7. catboost

In [ ]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 MB 2.2 MB/s eta 0:00:00


In [ ]:
from catboost import CatBoostClassifier

catboost_model = CatBoostClassifier()
catboost_model.fit(x_train, y_train)
# 테스트 데이터를 사용하여 예측
catboost_pred = catboost_model.predict(x_test)
catboost_pred_proba = catboost_model.predict_proba(x_test)

Learning rate set to 0.006596
0:	learn: 0.6857282	total: 54.3ms	remaining: 54.3s
1:	learn: 0.6779754	total: 59.1ms	remaining: 29.5s
2:	learn: 0.6713178	total: 64.1ms	remaining: 21.3s
3:	learn: 0.6635326	total: 68.7ms	remaining: 17.1s
4:	learn: 0.6555387	total: 74ms	remaining: 14.7s
5:	learn: 0.6475021	total: 79.2ms	remaining: 13.1s
6:	learn: 0.6400875	total: 84ms	remaining: 11.9s
7:	learn: 0.6336503	total: 88.5ms	remaining: 11s
8:	learn: 0.6271778	total: 93.2ms	remaining: 10.3s
9:	learn: 0.6202418	total: 97.9ms	remaining: 9.69s
10:	learn: 0.6141553	total: 102ms	remaining: 9.19s
11:	learn: 0.6068845	total: 107ms	remaining: 8.79s
12:	learn: 0.6002424	total: 111ms	remaining: 8.45s
13:	learn: 0.5952214	total: 116ms	remaining: 8.16s
14:	learn: 0.5883894	total: 120ms	remaining: 7.91s
15:	learn: 0.5818924	total: 125ms	remaining: 7.68s
16:	learn: 0.5762646	total: 129ms	remaining: 7.48s
17:	learn: 0.5701683	total: 134ms	remaining: 7.3s
18:	learn: 0.5643415	total: 138ms	remaining: 7.14s
19:	lear

In [ ]:
print("catboost")
get_clf_eval(y_test, catboost_pred)

catboost
오차행렬
[[1304  427]
 [   3   16]]
정확도: 0.7543, 정밀도 : 0.0361, 재현율:0.8421,F1 스코어:0.0693
ROC 스코어: 0.7977, PR score : 0.0321


In [ ]:
from sklearn.model_selection import GridSearchCV
from catboost import CatBoostClassifier

# 그리드서치를 위한 하이퍼파라미터 그리드 설정
param_grid = {
    'iterations': [100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1, 0.3],
    'depth': [3, 5, 7],
    'l2_leaf_reg': [1, 3, 5]
}

# CatBoostClassifier 모델 생성
catboost_clf = CatBoostClassifier()

# 그리드서치 수행
grid_search = GridSearchCV(catboost_clf, param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(x_train, y_train)

# 최적의 모델과 하이퍼파라미터 가져오기
best_catboost_clf = grid_search.best_estimator_

# 테스트 데이터에 대한 예측 수행
catboost_pred = best_catboost_clf.predict(x_test)
catboost_pred_proba = best_catboost_clf.predict_proba(x_test)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
200:	learn: 0.1510512	total: 1.73s	remaining: 853ms
201:	learn: 0.1506077	total: 1.74s	remaining: 844ms
202:	learn: 0.1502140	total: 1.75s	remaining: 836ms
203:	learn: 0.1495522	total: 1.76s	remaining: 827ms
204:	learn: 0.1489926	total: 1.76s	remaining: 818ms
205:	learn: 0.1483073	total: 1.77s	remaining: 809ms
206:	learn: 0.1478013	total: 1.78s	remaining: 801ms
207:	learn: 0.1473798	total: 1.79s	remaining: 792ms
208:	learn: 0.1466682	total: 1.8s	remaining: 783ms
209:	learn: 0.1460864	total: 1.81s	remaining: 774ms
210:	learn: 0.1453520	total: 1.81s	remaining: 765ms
211:	learn: 0.1448474	total: 1.82s	remaining: 756ms
212:	learn: 0.1441389	total: 1.83s	remaining: 747ms
213:	learn: 0.1436369	total: 1.84s	remaining: 738ms
214:	learn: 0.1432210	total: 1.84s	remaining: 730ms
215:	learn: 0.1428564	total: 1.85s	remaining: 721ms
216:	learn: 0.1424819	total: 1.86s	remaining: 712ms
217:	learn: 0.1420751	total: 1.87s	remaining: 703ms
218:	learn: 0.1413834	total: 

In [ ]:
# 임계값 설정
threshold = 0.28
catboost_pred = (catboost_pred_proba[:, 1] > threshold).astype(int)

# 성능 평가
print("catboost")
get_clf_eval(y_test, catboost_pred)

catboost
오차행렬
[[1236  495]
 [   1   18]]
정확도: 0.7166, 정밀도 : 0.0351, 재현율:0.9474,F1 스코어:0.0677
ROC 스코어: 0.8307, PR score : 0.0338
